# Desafio Hurb
---

<div style="text-align: right"> Por: Amanda Oliveira </div>

### Lidando com cancelamento de reservas  


Problemas propostos:

1. Descoberta de padrões úteis nos dados históricos de reservas que podem ser utilizados na formulação de melhores estratégias comerciais para lidar com cancelamentos.  

2. Construção de um modelo capaz de prever se uma reserva será cancelada.  


## Avaliação dos dados

Primeiramente, é importante que se tenha uma noção dos dados com os quais se está lidando. No caso, o dataset vem do artigo [**Hotel booking demand datasets**](https://www.sciencedirect.com/science/article/pii/S2352340918315191), de Nuno Antônio, Ana de Almeida e Luis Nunes, publicado em _Data in Brief_ em 2019.

Decidi usar a biblioteca Pandas para manusear os dados, uma vez que ela já tem embutidas diversas ferramentas de análise. Comecei verificando quais as variáveis disponíveis e que tipo de variável são, além de uma visão superficial dos valores que poderiam assumir.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ppscore as pps
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import StratifiedKFold

In [2]:
reservas = pd.read_csv("hotel_bookings.csv")
reservas

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [3]:
reservas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

Verificadas as variáveis disponíveis, a primeira plot gerada foi a matriz de correlação. 

A despeito de não termos muitos padrões claros de correlação com o cancelamento das reservas, pelo gráfico podemos perceber que existem alguns padrões que podemos observar:

- Correlação positiva com relação ao tempo entre a reserva e a data de chegada (``lead_time``). O gráfico mostra que uma reserva feita com maior antecedência tem maior chance de ser cancelada.

- Correlação positiva com relação a cancelamentos passados (``previous_cancelations``). Não é algo inesperado de se assumir que alguém que já cancelou no passado tenha maior chance de cancelar novamente uma reserva.

- Correlação negativa com relação a mudanças na reserva (``booking_changes``). Novamente, seria de se esperar que se a pessoa realizou mudanças na reserva, ela tem menor chance de cancelá-la.

- Correlação negativa com relação à necessidade de vagas para carro (``required_car_parking_spaces``). Pedidos com necessidades maiores de vagas têm menos chance de serem cancelados.

- Correlação negativa com relação a pedidos especiais (``total_of_special_requests``). É interessante observar que reservas com pedidos especiais têm menor chance de serem canceladas do que reservas genéricas.

Dentre esses, ``lead_time`` e ``total_of_special_requests`` se mostram como as variáveis com mais fortes correlações (positiva e negativa, respectivamente) com o cancelamento de reservas.

In [4]:
corr = reservas.corr()

cmap = cm.seismic
cmap._i_under = -1

corr.style.background_gradient(cmap=cmap)

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
is_canceled,1.000000,0.293123,0.016660,0.008148,-0.006130,-0.001791,0.024765,0.060017,0.005048,-0.032491,-0.084793,0.110133,-0.057358,-0.144381,-0.083114,-0.020642,0.054186,0.047557,-0.195498,-0.234658
lead_time,0.293123,1.000000,0.040142,0.126871,0.002268,0.085671,0.165799,0.119519,-0.037622,-0.020915,-0.124410,0.086042,-0.073548,0.000149,-0.069741,0.151464,0.170084,-0.063077,-0.116451,-0.095712
arrival_date_year,0.016660,0.040142,1.000000,-0.540561,-0.000221,0.021497,0.030883,0.029635,0.054624,-0.013192,0.010341,-0.119822,0.029218,0.030872,0.063457,0.259095,-0.056497,0.197580,-0.013684,0.108531
arrival_date_week_number,0.008148,0.126871,-0.540561,1.000000,0.066809,0.018208,0.015558,0.025909,0.005518,0.010395,-0.030131,0.035501,-0.020904,0.005508,-0.031201,-0.076760,0.022933,0.075791,0.001920,0.026149
arrival_date_day_of_month,-0.006130,0.002268,-0.000221,0.066809,1.000000,-0.016354,-0.028174,-0.001566,0.014544,-0.000230,-0.006145,-0.027011,-0.000300,0.010613,0.001487,0.044858,0.022728,0.030245,0.008683,0.003062
stays_in_weekend_nights,-0.001791,0.085671,0.021497,0.018208,-0.016354,1.000000,0.498969,0.091871,0.045793,0.018483,-0.087239,-0.012775,-0.042715,0.063281,0.140739,0.066749,-0.054151,0.049342,-0.018554,0.072671
stays_in_week_nights,0.024765,0.165799,0.030883,0.015558,-0.028174,0.498969,1.000000,0.092976,0.044203,0.020191,-0.097245,-0.013992,-0.048743,0.096209,0.182382,0.182211,-0.002020,0.065237,-0.024859,0.068192
adults,0.060017,0.119519,0.029635,0.025909,-0.001566,0.091871,0.092976,1.000000,0.030447,0.018146,-0.146426,-0.006738,-0.107983,-0.051673,-0.035594,0.207793,-0.008283,0.230641,0.014785,0.122884
children,0.005048,-0.037622,0.054624,0.005518,0.014544,0.045793,0.044203,0.030447,1.000000,0.024030,-0.032859,-0.024730,-0.021072,0.048949,0.041066,0.030931,-0.033273,0.324854,0.056253,0.081745
babies,-0.032491,-0.020915,-0.013192,0.010395,-0.000230,0.018483,0.020191,0.018146,0.024030,1.000000,-0.008943,-0.007501,-0.006550,0.083440,0.036184,0.019206,-0.010621,0.029186,0.037383,0.097889


Essa análise, embora gere um panorama geral dos dados, não considera os relacionamentos de variáveis categóricas, que podem ser muito importantes. Por essa razão, decidi aplicar uma outra métrica, o _Predictive Power Score_ (PPS). Essa parece ser uma métrica boa para avaliar se há relacionamentos entre variáveis, pois ele tenta avaliar se existe algum tipo qualquer de relação entre elas, não só relações lineares. Ele retorna 1 se há uma relação perfeita e 0 se não há relação, de maneira assimétrica. No caso da implementação de PPS usada, os valores são calculados usando modelos árvores de decisão.

Podemos observar pelo gráfico que só a coluna ``reservation_status`` tem relacionamento direto com a variável de cancelamento. Esse é um caso já esperado, já que essa variável contém a informação de cancelamento em si mesma. Além dessa, só a variável ``lead_time`` apresenta, sozinha, poder de predição sobre o cancelamento. A mesma aprece tanto na métrica de correlação como no PPS, reforçando a sua importância relativa neste contexto.
Por sua vez, para a ``lead_time`` as variáveis ``market_segment``, ``deposit_type`` e ``company`` se mostraram como tendo as maiores capacidades de predição da mesma.

In [7]:
ppsmat = pps.matrix(reservas)

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selecti

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selecti

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selecti

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.9/site-packages/sklearn/model_selecti

In [9]:
matrix_df = ppsmat[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')
matrix_df.style.background_gradient(cmap="Reds")

x,adr,adults,agent,arrival_date_day_of_month,arrival_date_month,arrival_date_week_number,arrival_date_year,assigned_room_type,babies,booking_changes,children,company,country,customer_type,days_in_waiting_list,deposit_type,distribution_channel,hotel,is_canceled,is_repeated_guest,lead_time,market_segment,meal,previous_bookings_not_canceled,previous_cancellations,required_car_parking_spaces,reservation_status,reservation_status_date,reserved_room_type,stays_in_week_nights,stays_in_weekend_nights,total_of_special_requests
y,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
adr,1.000000,0.047214,0.114700,0.000000,0.107997,0.106743,0.009092,0.028123,0.000000,0.000000,0.028702,0.448781,0.000653,0.004836,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074086,0.000000,0.000000,0.000000,0.000000,0.000000,0.027446,0.064691,0.000000,0.000000,0.002749
adults,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.146084,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
agent,0.025181,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.303061,0.000000,0.000000,0.000000,0.000000,0.000000,0.495472,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
arrival_date_day_of_month,0.000000,0.000000,0.012018,1.000000,0.000000,0.406510,0.000000,0.000000,0.000000,0.000116,0.000000,0.169626,0.000000,0.000000,0.003548,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.205679,0.000000,0.001020,0.001540,0.000000
arrival_date_month,0.129990,0.000000,0.091848,0.000000,1.000000,0.910317,0.000000,0.000000,0.000000,0.000000,0.000000,0.297504,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.581502,0.000000,0.000000,0.000000,0.000000
arrival_date_week_number,0.000000,0.001132,0.045331,0.001614,0.901474,1.000000,0.171170,0.000000,0.000000,0.000000,0.000316,0.256756,0.000000,0.004106,0.008678,0.000000,0.000000,0.000000,0.000000,0.000556,0.009764,0.002887,0.003374,0.001059,0.006138,0.000000,0.000214,0.540160,0.000000,0.002704,0.001710,0.000000
arrival_date_year,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.225127,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.829086,0.000000,0.000000,0.000000,0.000000
assigned_room_type,0.182006,0.127641,0.101804,0.000000,0.000000,0.003479,0.000000,1.000000,0.001697,0.001033,0.034837,0.179595,0.006560,0.000000,0.002252,0.000436,0.000000,0.000000,0.000000,0.000000,0.019316,0.000323,0.000000,0.000946,0.000000,0.008569,0.000000,0.026931,0.748631,0.001152,0.001862,0.001786
babies,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Feita uma análise inicial dos dados, o próximo passo é a própria criação de modelos de predição de cancelamento de reservas.

Neste caso, decidi aplicar (pelo menos à princípio) duas técnicas: Árvores de Decisão, que são modelos relativamente simples e podem ajudar na intuição dos dados; e Redes Neurais MLP, um modelo clássico de classificação que tem grande poder de generalização.

## Pré-processamento

Antes de se aplicar os modelos, é preciso fazer alguns ajustes nos dados. 

Primeiramente, separei a coluna ``is_canceled``, que equivale ao próprio rótulo de referência que queremos obter. Além disso é essencial retirar dos dados a coluna ``reservation_status``, já que esta já contém informação do cancelamento em si só e concede uma "informação futura" para o modelo. Também retirei a coluna ``reservation_status_date``, já que ela diz respeito à informação em ``reservation_status``, que foi retirada, e não agrega valor aos dados restantes. 

Para poderem ser aplicados aos modelos, os dados categóricos foram convertidos em numéricos através de "LabelEnconders". Os encoders foram salvos no dicionário ``encoders`` para que se possa fazer o mapeamento de volta para a variável catogórica, se desejado.

In [12]:
# Pre-processamento

input_data = reservas.copy()
labels = input_data.pop("is_canceled")

input_data.drop("reservation_status", inplace=True, axis=1)          # Essa coluna já tem o resultado, não é  entrada
input_data.drop("reservation_status_date", inplace=True, axis=1)

feature_name=list(input_data.columns)
class_name=["Ok", "Canceled"]

encoders = {}

for col in input_data.columns:
    if input_data[col].dtype == 'object':
        enc = LabelEncoder()
        
        try:
            enc.fit(input_data[col])
        except TypeError:
            input_data[col].fillna("No Data", inplace = True)
            enc.fit(input_data[col])
            
        input_data[col] = enc.transform(input_data[col])
        encoders[col] = enc
        
    elif input_data[col].isnull().values.any():
        input_data[col].fillna(-1, inplace = True)
        

input_data.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,1,342,2015,5,27,1,0,0,2,0.0,...,2,3,0,-1.0,-1.0,0,2,0.0,0,0
1,1,737,2015,5,27,1,0,0,2,0.0,...,2,4,0,-1.0,-1.0,0,2,0.0,0,0
2,1,7,2015,5,27,1,0,1,1,0.0,...,2,0,0,-1.0,-1.0,0,2,75.0,0,0
3,1,13,2015,5,27,1,0,1,1,0.0,...,0,0,0,304.0,-1.0,0,2,75.0,0,0
4,1,14,2015,5,27,1,0,2,2,0.0,...,0,0,0,240.0,-1.0,0,2,98.0,0,1


## Árvore de decisão

O primeiro modelo testado é o de árvores de decisão, um modelo relativamente simples que costuma ser útil para gerar uma intuição acerca dos dados, além de ser bem versátil para trabalhar com dados categóricos.

O conjunto de dados foi separado inicialmente em dois pacotes para a validação cruzada, e foram treinadas duas árvores, cada uma utilizando um dos pacotes para treino. O conjunto não utilizado para treino, em cada caso, foi utilizado para avaliar as árvores.

Para avaliação das árvores foi usada a métrica de acurácia (quantidade de acertos com relação ao total de eventos classificados) encima do conjunto de teste. Os resultados preliminares apresentaram cerca de 84,5% de acurácia.

In [13]:
from sklearn import tree

In [14]:
# Separation into training and testing set

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=0)     
# Determinado 'random_state' para que o resultado seja reproduzível

# Training
resulting_trees = []
tIdx = 1

for train_idx, test_idx in skf.split(input_data, labels):
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(input_data.iloc[train_idx], labels.iloc[train_idx])
    resulting_trees += [clf]
    
    # Preliminary results:
    print("Decision tree {}: ".format(tIdx))
    #r = tree.export_text(clf, feature_names=list(input_data.columns))
    #print(f"\t{r}")
    
    correctAnswers = 0
    totalAnswers = 0
    for idx in test_idx:
        result = clf.predict(np.array(input_data.iloc[idx]).reshape(1, -1))
        
        if result == labels.iloc[idx]:
            correctAnswers += 1
        totalAnswers += 1
    print(f"\tCorrect results in {100*float(correctAnswers)/totalAnswers}% of test set")
    
    tIdx += 1


Decision tree 1: 
	Correct results in 84.80107211659268% of test set
Decision tree 2: 
	Correct results in 85.03727280341737% of test set


A análise do processo de decisão das árvores se mostrou complicado demais para que se possa fazer inferências com facilidade. A despeito disso, alguns padrões recorrentes podem ser observados, como por exemplo: uma reserva com depósito do tipo 0 (_No Deposit_), ``lead_time`` pequeno (menos de 8.5 ou menos de 11.5, para cada uma das árvores) e que requer pelo menos uma vaga de estacionamento é considerado um caso no qual não haverá cancelamento.

A seguir está a descrição textual das duas árvores geradas, mostrando uma profundidade de até 5 estágios.

In [15]:
r = tree.export_text(resulting_trees[0], feature_names=list(input_data.columns), max_depth=5)
print(r)

|--- deposit_type <= 0.50
|   |--- lead_time <= 8.50
|   |   |--- required_car_parking_spaces <= 0.50
|   |   |   |--- country <= 134.50
|   |   |   |   |--- market_segment <= 5.50
|   |   |   |   |   |--- previous_cancellations <= 0.50
|   |   |   |   |   |   |--- truncated branch of depth 19
|   |   |   |   |   |--- previous_cancellations >  0.50
|   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |--- market_segment >  5.50
|   |   |   |   |   |--- total_of_special_requests <= 0.50
|   |   |   |   |   |   |--- truncated branch of depth 18
|   |   |   |   |   |--- total_of_special_requests >  0.50
|   |   |   |   |   |   |--- truncated branch of depth 18
|   |   |   |--- country >  134.50
|   |   |   |   |--- stays_in_weekend_nights <= 1.50
|   |   |   |   |   |--- previous_bookings_not_canceled <= 0.50
|   |   |   |   |   |   |--- truncated branch of depth 29
|   |   |   |   |   |--- previous_bookings_not_canceled >  0.50
|   |   |   |   |   |   |--- truncated b

In [16]:
r = tree.export_text(resulting_trees[1], feature_names=list(input_data.columns), max_depth=5)
print(r)

|--- deposit_type <= 0.50
|   |--- lead_time <= 11.50
|   |   |--- required_car_parking_spaces <= 0.50
|   |   |   |--- lead_time <= 7.50
|   |   |   |   |--- country <= 132.00
|   |   |   |   |   |--- agent <= 8.50
|   |   |   |   |   |   |--- truncated branch of depth 11
|   |   |   |   |   |--- agent >  8.50
|   |   |   |   |   |   |--- truncated branch of depth 22
|   |   |   |   |--- country >  132.00
|   |   |   |   |   |--- previous_bookings_not_canceled <= 0.50
|   |   |   |   |   |   |--- truncated branch of depth 27
|   |   |   |   |   |--- previous_bookings_not_canceled >  0.50
|   |   |   |   |   |   |--- truncated branch of depth 16
|   |   |   |--- lead_time >  7.50
|   |   |   |   |--- previous_cancellations <= 0.50
|   |   |   |   |   |--- market_segment <= 5.50
|   |   |   |   |   |   |--- truncated branch of depth 24
|   |   |   |   |   |--- market_segment >  5.50
|   |   |   |   |   |   |--- truncated branch of depth 16
|   |   |   |   |--- previous_cancellations >  

In [21]:
encoders["deposit_type"].inverse_transform([0])

array(['No Deposit'], dtype=object)

Como as árvores podem ter variação de desempenho de acordo com a profundidade que se permite que elas atinjam (árvores muito profundas tendem a perder poder de generalização), um experimento com variação da profundidade das árvores foi realizado, forçando a profundidade máxima a em 40, 30, 20, 10 e 5 estágios. Dessa vez o conjunto de dados foi separado em 5 pacotes para a validação cruzada.

Pode-se observar que a redução da profundidade das árvores apresentou um ganho até uma profundidade de 20 estágios, quando atingiu desempenho máximo (de aproximadamente 86%). Com uma diminuição maior da mesma houve perda de desempenho, ficando claro que para 10 ou 5 estágios a árvore não tem complexidade o suficiente para o problema.

In [107]:
# Separation into training and testing set
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)  

# Varying depth
depths = [40, 30, 20, 10, 5]
r_depth_trees = {}


for depth in depths:
    r_depth_trees[depth] = []
    tIdx = 1
    
    print("Maximum depth: {}".format(depth))
    
    for train_idx, test_idx in skf.split(input_data, labels):
        clf = tree.DecisionTreeClassifier(max_depth=depth)
        clf = clf.fit(input_data.iloc[train_idx], labels.iloc[train_idx])
        r_depth_trees[depth] += [clf]
        
        print("\tDecision tree {}: (depth of {})".format(tIdx, clf.get_depth()))
        
        correctAnswers = 0
        totalAnswers = 0
        for idx in test_idx:
            result = clf.predict(np.array(input_data.iloc[idx]).reshape(1, -1))
            if result == labels.iloc[idx]:
                correctAnswers += 1
            totalAnswers += 1
        print(f"\t\tCorrect results in {100*float(correctAnswers)/totalAnswers}% of test set")
        
        tIdx += 1


Maximum depth: 40
	Decision tree 1: (depth of 40)
		Correct results in 85.69454332258469% of test set
	Decision tree 2: (depth of 40)
		Correct results in 85.27096071697797% of test set
	Decision tree 3: (depth of 40)
		Correct results in 85.81958287963816% of test set
	Decision tree 4: (depth of 40)
		Correct results in 85.88240221124047% of test set
	Decision tree 5: (depth of 40)
		Correct results in 85.73941449930896% of test set
Maximum depth: 30
	Decision tree 1: (depth of 30)
		Correct results in 85.67779220235353% of test set
	Decision tree 2: (depth of 30)
		Correct results in 85.42172711282352% of test set
	Decision tree 3: (depth of 30)
		Correct results in 85.72744785995476% of test set
	Decision tree 4: (depth of 30)
		Correct results in 85.95359745372309% of test set
	Decision tree 5: (depth of 30)
		Correct results in 85.88599907861122% of test set
Maximum depth: 20
	Decision tree 1: (depth of 20)
		Correct results in 86.12169688847942% of test set
	Decision tree 2: (dep

## Rede Neural (MLP)

Depois de aplicar árvores de decisão, o próximo modelo considerado foi o de redes neurais, mais especificamente de Multi Layer Perceptrons (MLP). Esse é um modelo com alto poder de generalização, tendo sido aplicado a diversos problemas, razão pela qual decidi testá-lo para este conjunto de dados.

Novamente, os modelos foram avaliados com a métrica de acurácia, para comparação com os resultados obtidos nas árvores de decisão.

Foram testados diferentes números de neurônios nas camadas do MLP, e MLPs com 1 e 2 camadas escondidas.

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [25]:
sizes = [[10, 1], [50, 1], [50, 50, 1], [100, 50, 1]]

# Separation into training and testing set
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Training
resulting_nets = []
tIdx = 1

feature_name=list(input_data.columns)
inputDim = len(feature_name)

for train_idx, test_idx in skf.split(input_data, labels):
    
    for s in sizes:
        print("\tNeural Network {} - sizes {}:".format(tIdx, s))
    
        m = Sequential()
        
        m.add(Input(shape=(inputDim,)))
        for l_s in s:
            m.add(Dense(l_s, activation='sigmoid'))

        m.compile(loss="mse", optimizer="RMSprop", metrics=["accuracy"])

        history = m.fit( 
                        np.asarray(input_data.iloc[train_idx]), np.asarray(labels.iloc[train_idx]),
                        validation_data=( np.asarray(input_data.iloc[test_idx]), np.asarray(labels.iloc[test_idx]) ) 
                       )

        #a, accuracy = m.evaluate(np.asarray(input_data.iloc[test_idx]), np.asarray(labels.iloc[test_idx]))
        #print(f"\t\tCorrect results in {100*accuracy}% of test set")
        #print(a)
    
        tIdx += 1


	Neural Network 1 - sizes [10, 1]:
2985/2985 [==============================] - 8s 3ms/step - loss: 0.2351 - accuracy: 0.6296 - val_loss: 0.2332 - val_accuracy: 0.6296
	Neural Network 2 - sizes [50, 1]:
2985/2985 [==============================] - 7s 2ms/step - loss: 0.1927 - accuracy: 0.7213 - val_loss: 0.1820 - val_accuracy: 0.7413
	Neural Network 3 - sizes [50, 50, 1]:
2985/2985 [==============================] - 9s 3ms/step - loss: 0.1892 - accuracy: 0.7215 - val_loss: 0.1809 - val_accuracy: 0.7387
	Neural Network 4 - sizes [100, 50, 1]:
2985/2985 [==============================] - 10s 3ms/step - loss: 0.1864 - accuracy: 0.7286 - val_loss: 0.1744 - val_accuracy: 0.7585
	Neural Network 5 - sizes [10, 1]:
2985/2985 [==============================] - 7s 2ms/step - loss: 0.2034 - accuracy: 0.7011 - val_loss: 0.1920 - val_accuracy: 0.7236
	Neural Network 6 - sizes [50, 1]:
2985/2985 [==============================] - 6s 2ms/step - loss: 0.1943 - accuracy: 0.7093 - val_loss: 0.1956 - val

Os resultados apresentados, ainda que iniciais, mostram que a técnica de árvores de decisão se apresenta como uma solução mais adequada a este problema. Acredito que o fato de termos variáveis categóricas nos dados, para as quais dois valores numéricos próximos não significam que os exemplos são mais semelhantes do que outros que têm valores numéricos mais distintos, possa atrapalhar o desempenho da rede neste caso.

De qualquer forma, a diferença de acurácia é bem grande (aproximadamente 10 pontos percentuais). Foram feitos testes com variações do número de neurônios, mas em nenhum caso os resultados chegaram na casa dos 80% de acurácia no conjunto de teste, enquanto as árvores de decisão conseguiram desempenho de 86% com alterações na profundidade da mesma. 

Considerando os resultados, decidi testar um terceiro modelo, o de **AdaBoost**, para este conjunto de dados.

## AdaBoost

Esta técnica se baseia em ajustar um modelo inicial para um conjunto de dados, e depois ajustar cópias do modelo modificadas para focar em casos mais difíceis. Por essa razão, decidi aplicar o mesmo utilizando a árvore de decisão como base, tentando ver se este método consegue melhorar o desempenho original.

A mesma métrica de acurácia foi utilizada, e a mesma validação cruzada anterior.

In [111]:
from sklearn.ensemble import AdaBoostClassifier

In [122]:
# Separation into training and testing set
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)  

# Training
best_depth = 20      # Se mostrou o melhor resultado nos experimentos
resulting_boosts = []
idx = 1

for train_idx, test_idx in skf.split(input_data, labels):
    clf = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=best_depth))
    clf.fit(input_data.iloc[train_idx], labels.iloc[train_idx])
    resulting_boosts += [clf]
    
    print("AdaBoost {}:".format(tIdx))
    
    correctAnswers = 0
    totalAnswers = 0
    for idx in test_idx:
        result = clf.predict(np.array(input_data.iloc[idx]).reshape(1, -1))
        if result == labels.iloc[idx]:
            correctAnswers += 1
        totalAnswers += 1
    print(f"\tCorrect results in {100*float(correctAnswers)/totalAnswers}% of test set")
    
    tIdx += 1



AdaBoost 1:
	Correct results in 86.75405167720591% of test set
AdaBoost 2:
	Correct results in 86.5734148588659% of test set
AdaBoost 3:
	Correct results in 86.79956445263423% of test set
AdaBoost 4:
	Correct results in 86.6739257894296% of test set
AdaBoost 5:
	Correct results in 86.98747748879676% of test set


Este modelo foi capaz de melhorar ligeiramente o desempenho das árvores de decisão. Os resultados das árvores estavam em 86.12 +- 0.13 enquanto o AdaBoost teve resultados 86.76 +- 0.14. É uma melhora acurácia, mas o modelo desenvolvido ficou bem mais lento do que o original, já que tem que passar por várias árvores.

Neste caso, a escolha do modelo a ser usado fica a critério do que é mais relevante para o usuário. Um usuário que deseja classificar milhões de reservas por dia pode preferir o modelo mais rápido, priorizando a diminuição do custo computacional, enquanto aquele que tem um conjunto relativamente pequeno de reservas para avaliar pode julgar que vale mais a pena usar o método com desempenho superior. Além disso, a escolha depende também das necessidades de cada caso, pois para o usuário um certo patamar de identificação de cancelamentos pode ser o suficiente, ou ele pode ter condições extremas nas quais deseja extrair o máximo possível. 

Levando em consideração que a escolha do modelo vai ter muita influência da natureza da necessidade do usuário neste caso decidi selecionar a técnica inicial, de árvores de decisão, para ser o classificador final.

# Modelo de predição final

Tendo em vista os modelos testados, decidi utilizar como classificador a técnica de árvores de decisão. Embora já tenham sido treinadas árvores que servem para fazer a predição de cancelamento neste notebook, é importante que mais métricas que apenas a acurácia sejam analisados, e que seja dada mais informação final do que a predição de se a reserva será cancelada ou não. Além disso, neste treinamento final acrescentei uma quantidade de repetições do treinamento das árvores, no caso 10 repetições para cada pacote, e dessas 10 é selecionada uma das árvores, a de melhor acurácia balanceada (explicada mais abaixo). Dessa forma são levandas em conta as flutuações estatísticas do treinamento. No fim, obtermos um conjunto de 5 árvores, uma para cada pacote da validação cruzada.

É importante que a análise do modelo aborde as relações de falsos positivos e falsos negativos. Precisamos saber não só quantas predições se acerta, mas também qual a porcentagem de erro de classificação de reservas canceladas e de reservas não canceladas, erros esses que podem ter diferentes graus de importância em um dado problema. No caso de cancelamento de reservas, minha intuição me diz que é melhor que se classifique uma reserva não cancelada como cancelada do que uma reserva cancelada como não sendo cancelada. Considerando que os hoteis desejam dar atenção aos casos de maior risco, eu diria que é melhor colocar mais casos no grupo de risco em potencial.

Para avaliar essas características no modelo, as métricas utilizadas fpram a acurácia, acurácia balanceada, precision e recall. As mesmas estão explicadas a seguir:

- A acurácia, como já foi explicado, corresponde à taxa de acerto da classificação, ou seja, à quantidade de reservas corretamente classificadas com relação ao total de reservas.

- A acurácia balanceada corresponde à média aritmética entre a taxa de acertos das reservas canceladas e a taxa de acerto das reservas não canceladas. Se há muito mais casos de um determinado tipo, a acurácia fica tendenciosa para a classificação mais comum (no caso as reservas não canceladas), o que é evitado na acurácia balanceada.

- Precision equivale à taxa de acertos com relação às reservas classificadas como cancelamentos. Ou seja, equivale à porcentagem das reservas ditas como canceladas que de fato foram canceladas.

- Recall é a taxa de acertos com relação ao total de reservas canceladas. Ou seja, ele é a parcentagem das reservas canceladas que de fato foram classificadas como canceladas.

É evidente a importância dessas quatro métricas no problema. Em especial, é extremamente importante sabermos qual a "precisão" e qual o recall do classificador, para entendermos o quanto obtemos de reservas não canceladas e o quanto deixamos de obter de reservas canceladas com o nosso modelo.

In [28]:
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score

In [29]:
nSplits = 5
repeat = 10
best_depth = 20
mectrics = ['accuracy', 'balanced accuracy', 'precision', 'recall']


classifiers = []
score_metrics = {m:[] for m in mectrics}


skf = StratifiedKFold(n_splits=nSplits, shuffle=True, random_state=0)

tIdx = 1

for train_idx, test_idx in skf.split(input_data, labels):
    bestBAcc = 0
    bestClf = None
    best_predictions = None
    
    for r in range(repeat):
        clf = tree.DecisionTreeClassifier(max_depth=best_depth)
        clf = clf.fit(input_data.iloc[train_idx], labels.iloc[train_idx])

        prediction = clf.predict(np.array(input_data.iloc[test_idx]))
        
        score = balanced_accuracy_score(labels.iloc[test_idx], prediction)
        
        if score > bestBAcc:
            bestClf = clf
            bestBAcc = score
            best_predictions = prediction
    
    classifiers += [bestClf]
    score_metrics['accuracy'] += [accuracy_score(labels.iloc[test_idx], prediction)]
    score_metrics['balanced accuracy'] += [bestBAcc]
    score_metrics['precision'] += [precision_score(labels.iloc[test_idx], prediction)]
    score_metrics['recall'] += [recall_score(labels.iloc[test_idx], prediction)]
    
    print(f"Classifier {tIdx}: Balanced accuracy of {bestBAcc}")
        
    tIdx += 1

Classifier 1: Balanced accuracy of 0.8539458833759415
Classifier 2: Balanced accuracy of 0.8537092148921139
Classifier 3: Balanced accuracy of 0.8523529110274336
Classifier 4: Balanced accuracy of 0.8561036832591814
Classifier 5: Balanced accuracy of 0.8542027287470861


As médias e desvios padrão das métricas para os classificadores selecionados podem ser vistos a seguir.

Observamos que aa taxas não são uniformes entre si, o que já era esperado. A acurácia balanceada tem valor inferior à acurácia (tambpem esperado), mas de apenas 1 ponto percentual. 
A precisão e o recall são mais baixos, mas ainda conseguem bons valores, ambos com aproximadamente 81%. Isso significa que, das reservas classificadas como cancelamentos, aproximadamente 81 a cada 100 delas serão de fato canceladas e, similarmente, de cada 100 cancelamentos serão detectados aproximadamente 81 deles pelo detector.

In [30]:
for m in mectrics:
    arr = np.array(score_metrics[m])
    print(f"{m} score: {arr.mean()} +- {arr.std()}")
    

accuracy score: 0.8628109556914316 +- 0.0012156329713729238
balanced accuracy score: 0.8540628842603514 +- 0.0012039266356695649
precision score: 0.8126305680837937 +- 0.004821637333493665
recall score: 0.8184017652627965 +- 0.004964233404859571


Por fim, o classificador final pode ser usado com a função a seguir, ``predict_cancelation``, que retorna a predição de cancelamento (ou não) e a probabilidade de cancelamento prevista pelo mesmo.

A probabilidade equivale à proporção de árvores que classificam a reserva como um cancelamento com relação ao total das árvores, e a reserva será considerada um cancelamento quanto a maioria das árvores classifcar a mesma dessa forma (ou seja, quando a probabilidade de ser cancelamento for maior que 50%).

Este modelo pode ser ajustado, se desejado, para que a classificação utilize outro limiar de separação. Por exemplo, considerando que já será considerado um cancelamento quando a reserva tiver probabilidade de pelo menos 40% de ser cancelamento (se 2 das 5 árvores classificarem como cancelamento). Nesse caso aumentaríamos o recall (classificando uma maior porcentagem dos cancelamentos corretamente), o que pode ser que seja desejado mesmo em detrimento da precisão (haveria uma maior procentagem de reservas não canceladas classificadas como cancelamentos). 

Variando esse limiar, é possível ajustar a precisão e o recall para atender melhor as necessidades do usuário.

In [31]:
def predict_cancelation(df_in_data):
    cancel = 0
    not_cancel = 0
    
    for clf in classifiers:
        res = clf.predict(np.array(df_in_data))
        if res == 1:
            cancel += 1
        else:
            not_cancel += 1
    
    prediction = 1 if (cancel > not_cancel) else 0
    prob = float(cancel)/(cancel+not_cancel)
    
    # Retorna a classificação e a probabilidade de acerto
    return prediction, prob

In [35]:
# Resultado final (conjunto de dados completo)

VN, FP, FN, VP = 0, 0, 0, 0
for ind in input_data.index:
    v,p = predict_cancelation(np.array(input_data.iloc[ind]).reshape(1, -1))
    l = labels.iloc[ind]
    
    if l == 1:
        if v == 1:
            VP += 1
        else:
            FN += 1
    else:
        if v == 0:
            VN += 1
        else:
            FP += 1


print(f"Acurácia: {(VN+VP)/float(VN + FP + FN + VP)}")
print(f"Acurácia balanceada: {0.5*( VP/float(VP + FN) + VN/float(FP + VN) )}")
print(f"Precision: {VP/float(FP + VP)}")
print(f"Recall: {VP/float(FN + VP)}")

Acurácia: 0.931820085434291
Acurácia balanceada: 0.9266641819627358
Precision: 0.9089487284101727
Recall: 0.9067700795947902


No conjunto completo de dados, os resultados naturalmente ficam melhores, uma vez que estão incluidos dados que foram usados para fazer o próprio treinamento de cada modelo. O ideal é que neste momento seja utilizado um conjunto de validação, correspondendo a um dataset separado ou uma parte separada do próprio dataset original. No caso, não separei um conjunto de validação porque queria utilizar todos os dados para realizar o treinamento, e embora dessa forma não sobre dados que não foram usados no treinamento de nenhuma árvore para fazer essa classificação final, correspondendo a uma combinação das 5 árvores, a média e desvio padrão das métricas nos conjuntos de teste previamente apresentados já são o suficiente para dar uma noção da eficiência esperada do algoritmo.

Por fim, os resultados finais mostram uma boa discriminação das reservas canceladas, com ainda uma margem para ajuste das métricas d